In [1]:
!pip install -q --upgrade pip
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cpu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cpu.html


In [2]:
!pip uninstall jaxlib
!pip install -U jaxlib==0.4.2+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Found existing installation: jaxlib 0.4.2+cuda11.cudnn82
Uninstalling jaxlib-0.4.2+cuda11.cudnn82:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/jaxlib-0.4.2+cuda11.cudnn82.dist-info/*
    /usr/local/lib/python3.8/dist-packages/jaxlib/*
Proceed (Y/n)? Y
  Successfully uninstalled jaxlib-0.4.2+cuda11.cudnn82
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
  Using cached https://storage.googleapis.com/jax-releases/cuda11/jaxlib-0.4.2%2Bcuda11.cudnn82-cp38-cp38-manylinux2014_x86_64.whl (164.4 MB)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

#/content/drive/MyDrive/MasterarbeitData

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [7]:
from utils import *
from data_preperation import *
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
import os
import shutil

In [ ]:
dataset = TUDataset(root="Masterarbeit", name="MUTAG")

# make lists of dense adjacency matrix, node feature array and y
As = list()
graps_node_features = list()
ys = list()
for data in dataset:
    As.append(to_dense(data.edge_index, len(data.x)))
    graps_node_features.append(np.array(data.x))
    ys.append(np.array(data.y))

# save preprocessed data
# creates the following files in the base path directory
# ys.npy
# graps_node_features.npy
# graphs_edge_features.npy
# graph_conv_pattern.npy
# two_wl_pattern_radius_1.npy, two_wl_pattern_radius_2.npy, ..., two_wl_pattern_radius_k.npy
two_wl_radius = [1]
dataset_name = "MUTAG"
base_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/{dataset_name}"

if not os.path.exists(base_path):
    os.makedirs(base_path)

data_preperation(As, graps_node_features, ys, dataset_name, base_path, two_wl_radius)

In [22]:
# load preprocessed data

two_wl_radius = [1]
dataset_name = "MUTAG"
base_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/{dataset_name}"

ys = np.load(base_path + f"/ys.npy")
graps_node_features = np.load(base_path + f"/graps_node_features.npy")
graphs_edge_features = np.load(base_path + f"/graphs_edge_features.npy")
graph_conv_pattern = np.load(base_path + f"/graph_conv_pattern.npy")

two_wl_patterns = list()

for r, p in zip(two_wl_radius, two_wl_pattern):
    two_wl_patterns.append(np.load(base_path + f"/two_wl_pattern_radius_{r}.npy"))

print(As_1_pattern.shape)
print(As_3_pattern.shape)
print(graphs_edge_features.shape)

(18298, 4)
(353051, 4)
(188, 28, 28, 7)


In [14]:
# clear the folder
shutil.rmtree("/content/drive/MyDrive/MasterarbeitData/Preprocessed/splits/MUTAG")

In [28]:
# split the preprocessed data into pieces, to save memory in the kernel calcualtion

size = 10
r = 18
dataset_size = graphs_edge_features.shape[0]

base_path_splits = base_path + f"/splits/size_{size}_range_{r}" 
if not os.path.exists(base_path_splits):
    os.makedirs(base_path_splits)

for i in range(r+1):
  for j in range(r+1):
    
    if not i == r:
      x1_batch_range = list(range(i*size,(1+i)*size))
    else:
      x1_batch_range = list(range(i*size,dataset_size-1))

    if not j == r:
      x2_batch_range = list(range(j*size,(1+j)*size))
    else:
      x2_batch_range = list(range(j*size,dataset_size-1))
    
    np.save(base_path_splits + f"/graphs_edge_features_1_{i*size}_{(1+i)*size}", graphs_edge_features[x1_batch_range,:])
    np.save(base_path_splits + f"/graphs_edge_features_2_{j*size}_{(1+j)*size}", graphs_edge_features[x2_batch_range,:])

    for r, p in zip(two_wl_radius, two_wl_pattern):
      
      np.save(base_path_splits + f"/two_wl_pattern_radius_{r}_1_{i*size}_{(1+i)*size}", p[column_in_values(p[:,0], np.array(x1_batch_range)),:])
      np.save(base_path_splits + f"/two_wl_pattern_radius_{r}_2_{j*size}_{(1+j)*size}", p[column_in_values(p[:,0], np.array(x2_batch_range)),:])

